In [ ]:
import natsort
import numpy as np
import pandas as pd
import param
import plotly.express as px
import plotly.graph_objects as go
import re
import traceback
import panel as pn
pn.extension("plotly")
from io import BytesIO
from io import StringIO
from bokeh.models.widgets.tables import NumberFormatter
from sklearn.decomposition import PCA

In [ ]:
exec(open("class.py").read())
i_class = SpatialDataSet()

i_file = pn.widgets.FileInput(name='Upload file')

button_analysis = pn.widgets.Button(name='Analyse clusters', width=50)

i_acquisition = pn.widgets.Select(options=["SILAC", "LFQ"], name="Acquisition", width=300)
i_organism = pn.widgets.Select(options=["Human", "Mouse","Arabidopsis"], name="Organism", width=300)

i_clusterwidget = pn.widgets.Select(options=list(i_class.markerproteins), name="Cluster of interest", width=300)
i_mapwidget = pn.widgets.Select(options=[i_class.map_of_interest], name="Map of interest", width=300)

cache_uploaded = pn.widgets.Checkbox(value=False)
cache_run = pn.widgets.Checkbox(value=False)

analysis_status = pn.Pane("No analysis run yet")
filereading_status = "No data import yet"

i_expname = pn.widgets.TextInput(name='Experiment Name', placeholder='Enter your experiment name here here...')
#i_file = pn.widgets.Select(name='File',options=["6_deep_maps.txt","LFQ_proteinGroups.txt","proteinGroupsCOVID.txt", 
#                                              "proteinGroups_LFQ_Deep_3_Maps.txt"])
i_name_pattern = pn.widgets.Select(name='Name pattern',options=[".* (?P<cond>.*)_(?P<rep>.*)_(?P<frac>.*)",".* (?P<rep>.*)_(?P<frac>.*)",
                                                                ".* (?P<cond>.*)_(?P<frac>.*)_(?P<rep>.*)", "Custom"])
i_custom_namepattern = pn.widgets.TextInput(name='Customized Name Pattern', placeholder='Enter a string here...e.g. ".* (?P<rep>.*)_(?P<frac>.*)"')
regex_pattern = {
    ".* (?P<rep>.*)_(?P<frac>.*)" : ["MAP1_03K","MAP3_03K"],
    ".* (?P<cond>.*)_(?P<rep>.*)_(?P<frac>.*)" : ["EGF_rep1_06K","EGF_rep3_06K"],
    ".* (?P<cond>.*)_(?P<frac>.*)_(?P<rep>.*)" : ["Control_Mem_1", "Control_Cyt_1"]
    }
pattern_examples = pn.widgets.Select(name = "Examples", options=regex_pattern[i_name_pattern.value])

@pn.depends(i_name_pattern.param.value, i_custom_namepattern)
def custimization(name_pattern, custom_namepattern):
    if name_pattern == "Custom":
        return i_custom_namepattern
    else:
        example_for_name_pattern = regex_pattern[name_pattern]
        pattern_examples.options = example_for_name_pattern
        return pattern_examples



In [ ]:
i_class.name_pattern

In [ ]:
@pn.depends(i_file.param.value)
def read_file(file):
    if file is None:
        filereading_status = "No file is uploaded"
        cache_uploaded.value = False
        return ""
    else:
        cache_uploaded.value = False
        try:
            i_class.df_original = pd.read_csv(BytesIO(file), sep="\t", comment="#",
                                           usecols=lambda x: bool(re.match(i_class.regex["imported_columns"], x)),
                                           low_memory=False)     
            assert i_class.df_original.shape[0]>10 and i_class.df_original.shape[1]>5
            filereading_status = i_class.df_original.head()
            cache_uploaded.value = True
            i_acquisition.disabled = False
            #return pn.Column(pn.Pane(filereading_status, max_cols=10, width=20*filereading_status.shape[1]),
             #                i_acquisition,i_organism, button_analysis)
            return pn.Column(pn.Pane(filereading_status, max_cols=10, width=20*filereading_status.shape[1]),
                   pn.Row(i_acquisition, i_expname), pn.Row(i_name_pattern, custimization), pn.Row(button_analysis))

        except: 
            filereading_status = [traceback.format_exc(),file[0:50]]
            cache_uploaded.value = False
            return pn.Column(pn.Pane(filereading_status,width=200))     


def update_object_selector(i_mapwidget):
    i_mapwidget.options = list(i_class.map_names)
    if i_class.map_of_interest not in list(i_class.map_names):
            i_class.map_of_interest = i_class.map_names[0]

number_of_analayses = 0           
            
def execution(event):
    #prevent execution, if no data is uploaded yet
    if cache_uploaded.value == False:
        analysis_status.object = "Please upload a file first"
    else:        
       # number_of_analayses = number_of_analayses + 1
       # if number_of_analayses.pop(0)
        dashboard_analysis.objects = []
        cache_run.value = False
        i_acquisition.disabled = True
        try:
            #dashboard_analysis.pop(0)
            dashboard_analysis.append(i_clusterwidget)
            dashboard_analysis.append(i_mapwidget)
            dashboard_analysis.append(analysis_tabs)
            analysis_status.object = "Analysis in progress"
            i_class.acquisition = i_acquisition.value
            
            i_class.acquisition = i_acquisition.value
            if i_name_pattern.value == "Custom":
                i_class.name_pattern = i_custom_namepattern.value
            else:
                i_class.name_pattern = i_name_pattern.value
            
            #if not i_name_pattern.value == "Custom":
            #    i_class.name_pattern = i_name_pattern.value
            #else:
            #    i_class.name_pattern = i_custom_namepattern.value
            i_class.expname = i_expname.value
            i_class.processingdf()
            update_object_selector(i_mapwidget)
            i_class.perform_pca()
            i_class.multiple_iterations()
            i_class.distance_calculation()
            analysis_status.object = "Analysis finished! Please open the 'Analysis' tab!"
            cache_run.value = True
    
        except:
            i_acquisition.disabled = False
            
            analysis_status.object = "Analysis was not possible!"
            #analysis_status.object = [traceback.format_exc(), "Analyis was not possible"]
            cache_run.value = False

@pn.depends(i_clusterwidget.param.value,i_mapwidget.param.value, cache_run.param.value)
def update_cluster_overview(clusterwidget, mapwidget, run):
    try:
        if run == True:
            i_class.cluster_of_interest = clusterwidget
            i_class.map_of_interest = mapwidget
            list_genes = [goi for goi in i_class.genenames_sortedout_list if goi in i_class.markerproteins[clusterwidget]]
            cluster_overview = pn.Column(
                    pn.Row(pn.Pane(i_class.plot_pca(), width=500),
                           pn.Pane(i_class.distance_boxplot(), width=500),
                           pn.Pane(i_class.plottingdf(), width=500)),
                    pn.Row(
                        "The following proteins were not consistently quantified throughout all maps: {}".format(
                            ', '.join(list_genes)) if len(list_genes) != 0 else
                        "All genes from this cluster are quantified in all maps.")
                    )
            app_tabs.active = 1
            return cluster_overview
        else:
            cluster_overview = "Run analysis first!"
            return cluster_overview
    except:
        update_status = [traceback.format_exc(), "Analyis was not possible"]
        return update_status

    
@pn.depends(i_clusterwidget.param.value, cache_run.param.value)
def update_cluster_details(clusterwidget, run):
    try:
        if run == True:
            i_class.cluster_of_interest = clusterwidget
            cluster_details = i_class.distance_to_median_boxplot()
            return cluster_details
        else:
            cluster_details = "Run analysis first!"
            return cluster_details
    except:
        update_status = [traceback.format_exc(), "Analyis was not possible"]
        return update_status


@pn.depends(cache_run.param.value)
def show_tabular_overview(run):
    try:
        if run == True:
            content = pn.Column(
                pn.panel(i_class.results_overview_table(),width=500),
                pn.widgets.FileDownload(
                    callback=table_download, filename='cluster_distances.csv')
            )
            return content
        else:
            content = "Please, upload a file first and press ‘Analyse clusters’"
            return content
    except:
        content = [traceback.format_exc(), "Analyis was not possible"]
        return content

@pn.depends(cache_run.param.value)
def table_download(run):
    df = i_class.results_overview_table()
    sio = StringIO()
    df.to_csv(sio)
    sio.seek(0)
    return sio 

button_analysis.on_click(execution)  

dasboard_home = pn.Column(i_file, read_file,analysis_status, name="Home", css_classes=["content-width"])
dashboard_analysis = pn.Column("Please, upload a file first and press 'Analyse clusters'", name="Analysis", css_classes=["content-width"])

analysis_tabs = pn.Tabs(margin=10, css_classes=["content-width"], dynamic=True)
analysis_tabs.append(("Cluster Overview", update_cluster_overview))
analysis_tabs.append(("Cluster Details", update_cluster_details))

In [ ]:
app_tabs = pn.Tabs(margin=10, css_classes=["content-width"], dynamic=True)
app_tabs.append(("Home", dasboard_home))
app_tabs.append(("Analysis", dashboard_analysis))
app_tabs.append(("Download", show_tabular_overview))
app_tabs.append(("About", "Explanation of what's going on here"))


#i_search = pn.widgets.TextInput(name="Search")
app_center = pn.Column(pn.Row(pn.Pane("# QC tool for Spatial Proteomics"),
                              pn.layout.HSpacer(),
                              #i_search,
                              #width=1600, 
                              margin=10),
                       app_tabs,
                       pn.Spacer(background="#DDDDDD", height=100, margin=0))
app = pn.GridSpec(sizing_mode="stretch_both", margin=0)
app[0,0] = pn.Spacer(background="#DDDDDD", margin=0)
app[0,9] = pn.Spacer(background="#DDDDDD", margin=0)
app[0,1:8] = app_center

pwd = pn.widgets.PasswordInput(name="Please enter password for access.")
app_container = pn.Column(pwd)

def check_pwd(event, app=app):
    pwd = event.new
    if pwd == "pwd":
        app_container[0]=app
pwd.param.watch(check_pwd, 'value')

In [ ]:
 try:
     server.stop()
 except:
     print("First server startup")
 server = app.show(port=5063, websocket_max_message_size=1000000000)

In [ ]:
i_class.analysed_datasets_dict

In [ ]:
i_class.analysed_datasets_dict['l']['changes in Shape after filtering']

In [ ]:
i_class.analysed_datasets_dict['s']['changes in Shape after filtering']

In [ ]:
i_class.analysed_datasets_dict['cov']

In [ ]:
i_file

In [ ]:
i_file = 'LFQ_proteinGroups.txt'

In [ ]:
i_name_pattern.value = '.* (?P<rep>.*_.*)_(?P<frac>.*)'

In [ ]:
i_name_pattern.value

In [ ]:
['LFQ_proteinGroups.txt', 'LFQ', '.* (?P<rep>.*_.*)_(?P<frac>.*)', 'LFQ_1']